In [1]:
import fiona
import geopandas as gpd
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [2]:
pd.set_option('display.max_columns', None)  

# Data from 2021

In [4]:
gdf = gpd.read_file('../data/Natl_WI.gdb', layer='NationalWalkabilityIndex')

In [5]:
warnings.filterwarnings('ignore', message='A value is trying to be set on a copy of a slice from a DataFrame')

# Bringing in Data Profiles from Census

In [7]:
dp02 = pd.read_csv('../data/dp02_final_real.csv')

In [8]:
dp03 = pd.read_csv('../data/dp03_final_real.csv')

In [9]:
dp04 = pd.read_csv('../data/dp04_final_real.csv')

In [10]:
dp05 = pd.read_csv('../data/dp05_final.csv')

In [11]:
dp0203 = dp02.merge(dp03, on='csa_name', how='inner')

In [12]:
dp020304 = dp0203.merge(dp04, on='csa_name', how='inner')

In [13]:
dps = dp020304.merge(dp05, on='csa_name', how='inner')

In [14]:
dps = dps.rename(columns={'Median (dollars)':'Median_Home_Value', 'Median (dollars).1':'Median_Rent', 'Car, truck, or van -- drove alone %': 'Car_Drove_To_Work'})

In [15]:
dps = dps.dropna(subset=['csa_name'])

In [16]:
dps.columns = dps.columns.str.lower()

# NAICS data

In [18]:
business = pd.read_csv('../data/businesses_final.csv')

C:\Users\noahf\AppData\Local\Temp\ipykernel_25816\2716533220.py:1: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  business = pd.read_csv('../data/businesses_final.csv')


In [19]:
business = business.dropna(subset=['Geographic Area Name'])

In [20]:
csa_rename_map = {
    'Asheville-Brevard, NC CSA': 'Asheville-Marion-Brevard, NC',
    'Atlanta--Athens-Clarke County--Sandy Springs, GA CSA': 'Atlanta--Athens-Clarke County--Sandy Springs, GA-AL',
    'Bend-Redmond-Prineville, OR CSA': 'Bend-Prineville, OR',
    'Boise City-Mountain Home-Ontario, ID-OR CSA': 'Boise City-Mountain Home-Ontario, ID-OR',
    'Boston-Worcester-Providence, MA-RI-NH-CT CSA': 'Boston-Worcester-Providence, MA-RI-NH-CT',
    'Cape Coral-Fort Myers-Naples, FL CSA': 'Cape Coral-Fort Myers-Naples, FL',
    'Cleveland-Akron-Canton, OH CSA': 'Cleveland-Akron-Canton, OH',
    'Columbia-Orangeburg-Newberry, SC CSA': 'Columbia-Orangeburg-Newberry, SC',
    'Dayton-Springfield-Sidney, OH CSA': 'Dayton-Springfield-Kettering, OH',
    'Detroit-Warren-Ann Arbor, MI CSA': 'Detroit-Warren-Ann Arbor, MI',
    'Fargo-Wahpeton, ND-MN CSA': 'Fargo-Wahpeton, ND-MN',
    'Grand Rapids-Wyoming-Muskegon, MI CSA': 'Grand Rapids-Kentwood-Muskegon, MI',
    'Harrisburg-York-Lebanon, PA CSA': 'Harrisburg-York-Lebanon, PA',
    'Harrisonburg-Staunton-Waynesboro, VA CSA': 'Harrisonburg-Staunton, VA',
    'Hartford-West Hartford, CT CSA': 'Hartford-East Hartford, CT',
    'Jackson-Vicksburg-Brookhaven, MS CSA': 'Jackson-Vicksburg-Brookhaven, MS',
    'Johnson City-Kingsport-Bristol, TN-VA CSA': 'Johnson City-Kingsport-Bristol, TN-VA',
    'Lexington-Fayette--Richmond--Frankfort, KY CSA': 'Lexington-Fayette--Richmond--Frankfort, KY',
    'Lincoln-Beatrice, NE CSA': 'Lincoln-Beatrice, NE',
    'Memphis-Forrest City, TN-MS-AR CSA': 'Memphis-Forrest City, TN-MS-AR',
    'Myrtle Beach-Conway, SC-NC CSA': 'Myrtle Beach-Conway, SC-NC',
    'Philadelphia-Reading-Camden, PA-NJ-DE-MD CSA': 'Philadelphia-Reading-Camden, PA-NJ-DE-MD',
    'Pittsburgh-New Castle-Weirton, PA-OH-WV CSA': 'Pittsburgh-New Castle-Weirton, PA-OH-WV',
    'Portland-Vancouver-Salem, OR-WA CSA': 'Portland-Vancouver-Salem, OR-WA',
    'Pueblo-Cañon City, CO CSA': 'Pueblo-Cañon City, CO',
    'Rochester-Austin, MN CSA': 'Rochester-Austin, MN',
    'Rockford-Freeport-Rochelle, IL CSA': 'Rockford-Freeport-Rochelle, IL',
    'San Jose-San Francisco-Oakland, CA CSA': 'San Jose-San Francisco-Oakland, CA',
    "Spokane-Spokane Valley-Coeur d'Alene, WA-ID CSA": "Spokane-Spokane Valley-Coeur d'Alene, WA-ID"
}

In [21]:
business["Geographic Area Name"] = business["Geographic Area Name"].replace(csa_rename_map)

In [22]:
business = business.rename(columns={'Geographic Area Name':'csa_name'})

In [23]:
business["Number of establishments"] = pd.to_numeric(
    business["Number of establishments"].str.replace(",", ""), errors="coerce"
)

business["Sales, value of shipments, or revenue ($1,000)"] = pd.to_numeric(
    business["Sales, value of shipments, or revenue ($1,000)"].str.replace(",", ""), errors="coerce"
)

In [24]:
group_business = business.groupby(
    ["csa_name", "2017 NAICS code", "Meaning of NAICS code"], 
    as_index=False
).agg({
    "Number of establishments": "sum",
    "Sales, value of shipments, or revenue ($1,000)": "sum"
})

In [25]:
group_business = group_business.rename(columns={'Number of establishments':'num_establishments', 'Sales, value of shipments, or revenue ($1,000)':'revenue',
                              'Meaning of NAICS code':'sector'})

In [26]:
group_business = group_business[['csa_name', 'sector', 'num_establishments','revenue']]

In [27]:
group_business["revenue_in_dollars"] = group_business["revenue"] * 1000

In [28]:
group_business["revenue_per_establishment"] = (
    group_business["revenue_in_dollars"] / 
    group_business["num_establishments"]
)

In [29]:
group_business["revenue_per_establishment"] = group_business["revenue_per_establishment"].round(2)
pd.options.display.float_format = '{:,.2f}'.format

In [30]:
#Fully cleaned post aggregation. revenue_per_establishment is made by taking revenue multiplying by 1000
#And then dividing that number by the number of businesses in that sector
group_business = group_business[['csa_name', 'sector', 'revenue_per_establishment']]

# Cleaning walkability data

In [32]:
gdf.columns = gdf.columns.str.lower()

In [33]:
gdf = gdf.rename(columns = {'geoid20': 'cbg_fips', 'statefp':'state_fips', 'countyfp':'county_fips', 
                      'ac_land': 'total_land_acres', 'ac_unpr': 'unprotected_land_acres', 
                      'd4a':'distance_from_transit_stop', 'natwalkind':'walk_score'})

In [34]:
#noticed there's -99999 for distance to transit. These are placeholder values for no data available.. getting rid of them to perform avgs
gdf = gdf[gdf['distance_from_transit_stop'] != -99999.00]

### Adding: cbsa_total_pop, cbsa_avg_walk_score, cbsa_pop_density, avg_dist_to_transit, avg_worker_density

In [36]:
#Calculating total population of each CBSA
#Found .transform that allows for aggregation to be indexed
gdf['csa_pop'] = gdf.groupby('csa')['totpop'].transform('sum')

In [37]:
#Calculating avg_walk_score for each cbsa
gdf['csa_avg_walk_score'] = gdf.groupby('csa')['walk_score'].transform('mean')

In [38]:
#Calculting avg distance to transit for each cbsa
gdf['avg_dist_to_transit'] = gdf.groupby('csa')['distance_from_transit_stop'].transform('mean')

In [39]:
#Calculating total land acres for each cbsa
gdf['csa_total_land_acres'] = gdf.groupby('csa')['total_land_acres'].transform('sum')

In [40]:
#Calculating total workers for each cbsa
gdf['csa_total_workers'] = gdf.groupby('csa')['workers'].transform('sum')

In [41]:
#Calculating Population density. Converting acres to sq mile to make more of a compelling measurement
gdf['pop_density_sq_mile'] = gdf['csa_pop'] / (gdf['csa_total_land_acres'] / 640)

In [42]:
#Calculating Worker density. Also by Sq Mile
gdf['worker_density_sq_mile'] = gdf['csa_total_workers'] / (gdf['csa_total_land_acres'] / 640)

In [43]:
# Interested in seeing the distribution with small, medium, and large cities 
# Originally Calculated using 25th percentile, median, and 75th percentile
# That resulted in only getting 3 for nlargest for 'Large' cities so I tweaked it to catch more
def classify_csa_size(pop):
    if pop < 175000:
        return "Small"
    elif pop <= 500000:
        return "Medium"
    else:
        return "Large"

gdf["csa_size"] = gdf["csa_pop"].apply(classify_csa_size)

In [44]:
walkability = gdf[['cbg_fips', 'state_fips', 'county_fips', 'blkgrpce', 'csa',
       'csa_name', 'csa_avg_walk_score', 'csa_pop','csa_size', 'csa_total_land_acres', 'csa_total_workers', 'worker_density_sq_mile', 
       'pop_density_sq_mile', 'total_land_acres', 'unprotected_land_acres', 'totpop',
       'counthu', 'hh', 'workers', 'd3b', 'distance_from_transit_stop', 'avg_dist_to_transit', 'walk_score', 'geometry']]

In [45]:
walkability = gdf[['csa_name', 'csa_avg_walk_score', 'csa_size', 'pop_density_sq_mile', 'worker_density_sq_mile', 'avg_dist_to_transit', 'geometry']]

In [46]:
unique_csa = walkability.drop_duplicates('csa_name')

In [47]:
#Probably show a breakdown of this in tableau or power bi to support 
unique_csa['csa_size'].value_counts()

csa_size
Large     44
Small     35
Medium    31
Name: count, dtype: int64

In [48]:
the30 = unique_csa[unique_csa['csa_name'].isin(['Rockford-Freeport-Rochelle, IL','Jackson-Vicksburg-Brookhaven, MS', 'Johnson City-Kingsport-Bristol, TN-VA',
                                        'Myrtle Beach-Conway, SC-NC', 'Rochester-Austin, MN', 'Cape Coral-Fort Myers-Naples, FL', 'Lexington-Fayette--Richmond--Frankfort, KY',
                                        'Grand Rapids-Kentwood-Muskegon, MI', 'Columbia-Orangeburg-Newberry, SC', 'Dayton-Springfield-Kettering, OH', 'Memphis-Forrest City, TN-MS-AR',
                                        'Detroit-Warren-Ann Arbor, MI', 'Atlanta--Athens-Clarke County--Sandy Springs, GA-AL', 'Hartford-East Hartford, CT', 'Cleveland-Akron-Canton, OH',
                                        'Pueblo-Cañon City, CO', 'Bend-Prineville, OR', 'Harrisonburg-Staunton, VA', 'Asheville-Marion-Brevard, NC', 'Burlington-Fort Madison-Keokuk, IA-IL-MO',
                                        'Harrisburg-York-Lebanon, PA', 'Boise City-Mountain Home-Ontario, ID-OR', 'Lincoln-Beatrice, NE', "Spokane-Spokane Valley-Coeur d'Alene, WA-ID",
                                        'Fargo-Wahpeton, ND-MN', 'Portland-Vancouver-Salem, OR-WA', 'Boston-Worcester-Providence, MA-RI-NH-CT', 'Pittsburgh-New Castle-Weirton, PA-OH-WV',
                                        'San Jose-San Francisco-Oakland, CA', 'Philadelphia-Reading-Camden, PA-NJ-DE-MD'])].reset_index(drop=True)

# Merging all data

In [50]:
master = the30.merge(dps, on='csa_name', how='inner')

In [51]:
#Pivoting group_business so final data frame will be 30 rows corresponding to only 1 instance of each city
naics_pivot = group_business[['csa_name', 'sector', 'revenue_per_establishment']].copy()

pivoted_naics = naics_pivot.pivot_table(
    index='csa_name',
    columns='sector',
    values='revenue_per_establishment'
)

pivoted_naics = pivoted_naics.reset_index()

pivoted_naics.columns.name = None

final_merged_df = pd.merge(
    master,
    pivoted_naics,
    on='csa_name',
    how='left'
)

In [52]:
final_merged_df.columns = final_merged_df.columns.str.strip()

In [53]:
columns_to_convert = ["bachelor's degree or higher", 'unemployment rate %', 'car_drove_to_work', 'public transportation  %',
'walked %', 'all people %', 'no vehicles available', '1 vehicle available', '2 vehicles available', '3 or more vehicles available']

In [54]:
for col in columns_to_convert:
    
    if final_merged_df[col].dtype == 'object':
        
        final_merged_df[col] = pd.to_numeric(final_merged_df[col].str.rstrip('%'), errors='coerce')

In [55]:
final_merged_df = final_merged_df.rename(columns={
    "bachelor's degree or higher": "bachelor's degree or higher %", 
    'car_drove_to_work': 'car_drove_to_work %', 
    'no vehicles available': 'no vehicles available %', 
    'public transportation  %': 'public transportation %', 
    '1 vehicle available': '1 vehicle available %',
    '2 vehicles available': '2 vehicles available %', 
    '3 or more vehicles available': '3 or more vehicles available %',
    'all people %': 'poverty_rate %',
    'accommodation and food services': 'accommodation and food services revenue per establishment', 
    'arts, entertainment, and recreation': 'arts, entertainment, and recreation revenue per establishment',
    'other services (except public administration)': 'local services revenue per establishment', 
    'retail trade': 'retail trade revenue per establishment'})

### Top 5 Least Walkable Cities (Small -> Medium -> Large)

In [57]:
Bottom_5_Small = final_merged_df[final_merged_df['csa_size'] == 'Small'].nsmallest(5, 'csa_avg_walk_score').reset_index(drop=True)

In [58]:
Bottom_5_Medium = final_merged_df[final_merged_df['csa_size'] == 'Medium'].nsmallest(5, 'csa_avg_walk_score').reset_index(drop=True)

In [59]:
Bottom_5_Large = final_merged_df[final_merged_df['csa_size'] == 'Large'].nsmallest(5, 'csa_avg_walk_score').reset_index(drop=True)

### Top 5 Most Walkable Cities (Small -> Medium -> Large)

In [61]:
Top_5_Small = final_merged_df[final_merged_df['csa_size'] == 'Small'].nlargest(5, 'csa_avg_walk_score').reset_index(drop=True)

In [62]:
Top_5_Medium = final_merged_df[final_merged_df['csa_size'] == 'Medium'].nlargest(5, 'csa_avg_walk_score').reset_index(drop=True)

In [63]:
Top_5_Large = final_merged_df[final_merged_df['csa_size'] == 'Large'].nlargest(5, 'csa_avg_walk_score').reset_index(drop=True)

In [64]:
Top_5_Large['walkability_ranking'] = 'Top'
Top_5_Medium['walkability_ranking'] = 'Top'
Top_5_Small['walkability_ranking'] = 'Top'
Bottom_5_Large['walkability_ranking'] = 'Bottom'
Bottom_5_Medium['walkability_ranking'] = 'Bottom'
Bottom_5_Small['walkability_ranking'] = 'Bottom'

In [65]:
all_dfs = [Top_5_Large, Top_5_Medium, Top_5_Small, Bottom_5_Large, Bottom_5_Medium, Bottom_5_Small]
final = pd.concat(all_dfs, ignore_index=True)

In [66]:
final = final.rename(columns={
    "bachelor's degree or higher": "bachelor's degree or higher %", 
    'car_drove_to_work': 'car_drove_to_work %', 
    'no vehicles available': 'no vehicles available %', 
    'public transportation  %': 'public transportation %', 
    '1 vehicle available': '1 vehicle available %',
    '2 vehicles available': '2 vehicles available %', 
    '3 or more vehicles available': '3 or more vehicles available %',
    'all people %': 'poverty_rate %',
    'accommodation and food services': 'accommodation and food services revenue per establishment', 
    'arts, entertainment, and recreation': 'arts, entertainment, and recreation revenue per establishment',
    'other services (except public administration)': 'local services revenue per establishment', 
    'Retail trade': 'retail trade revenue per establishment'})

In [67]:
final.columns

Index(['csa_name', 'csa_avg_walk_score', 'csa_size', 'pop_density_sq_mile',
       'worker_density_sq_mile', 'avg_dist_to_transit', 'geometry',
       'bachelor's degree or higher %', 'unemployment rate %',
       'car_drove_to_work %', 'public transportation %', 'walked %',
       'poverty_rate %', 'mean travel time to work (minutes)',
       'median household income (dollars)', 'no vehicles available %',
       '1 vehicle available %', '2 vehicles available %',
       '3 or more vehicles available %', 'median_home_value', 'median_rent',
       'median age (years)', 'Accommodation and food services',
       'Arts, entertainment, and recreation',
       'Other services (except public administration)',
       'retail trade revenue per establishment', 'walkability_ranking'],
      dtype='object')

In [68]:
final = final[['csa_name','csa_size', 'csa_avg_walk_score', 'walkability_ranking', 'pop_density_sq_mile',
       'worker_density_sq_mile', 'avg_dist_to_transit', 'geometry',
       "bachelor's degree or higher %", 'unemployment rate %',
       'car_drove_to_work %', 'public transportation %', 'walked %',
       'poverty_rate %', 'mean travel time to work (minutes)',
       'median household income (dollars)', 'no vehicles available %',
       '1 vehicle available %', '2 vehicles available %',
       '3 or more vehicles available %', 'median_home_value', 'median_rent',
       'median age (years)', 'Accommodation and food services',
       'Arts, entertainment, and recreation',
       'Other services (except public administration)',
       'retail trade revenue per establishment']]

# Running correlations to see what relationships exist between walkability and all of the other data

In [70]:
numerical_df = final.select_dtypes(include=['number'])

In [71]:
correlation_matrix = numerical_df.corr()

In [72]:
walk_score_correlations = correlation_matrix['csa_avg_walk_score'].sort_values(ascending=False)

In [73]:
walk_score_correlations

csa_avg_walk_score                               1.00
walked %                                         0.45
median_home_value                                0.39
3 or more vehicles available %                   0.34
bachelor's degree or higher %                    0.29
median household income (dollars)                0.29
median_rent                                      0.28
public transportation %                          0.26
worker_density_sq_mile                           0.25
retail trade revenue per establishment           0.22
pop_density_sq_mile                              0.18
Arts, entertainment, and recreation              0.13
no vehicles available %                          0.03
Other services (except public administration)   -0.04
mean travel time to work (minutes)              -0.05
median age (years)                              -0.08
2 vehicles available %                          -0.11
Accommodation and food services                 -0.11
1 vehicle available %       

# QUESTION 1

# Which demographic, socioeconomic (e.g., income, poverty, education), and built environment factors (e.g., density, transit proximity) correlate with higher walkability scores at the CSA level?